# Trinity Captain (WOS Phase-2) Visualization

Portfolio-level analysis of the Trinity Captain Tier-2 composite strategy.

In [2]:
# Imports
import asyncio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import svr3
import os

# Styling
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## Configuration

In [ ]:

TOKEN = "58abd12edbde042536637bfba9d20d5faf366ef481651cdbb046b1c3b4f7bf7a97ae7a2e6e5dc8fe05cd91147c8906f8a82aaa1bb1356d8cb3d6a076eadf5b5a"

# Date range
START_DATE = 20241025000000  # Adjust as needed
END_DATE = 20241101000000

# Tier-2 query parameters
INDICATOR_NAME = 'TrinityCaptain'
GRANULARITY = 900  # 15 minutes
MARKET = 'DCE'
COMMODITY = 'COMPOSITE<00>'  # Tier-2 placeholder

print(f'Fetching {INDICATOR_NAME} data for {MARKET}/{COMMODITY}')
print(f'Date range: {START_DATE} to {END_DATE}')

Fetching WOS_Captain data for DCE/COMPOSITE<00>
Date range: 20241025000000 to 20241101000000


## Data Fetch

In [4]:
async def fetch_captain_data():
    '''Fetch Captain strategy data using svr3'''
    client = svr3.sv_reader(
        START_DATE,
        END_DATE,
        INDICATOR_NAME,
        GRANULARITY,
        'private',
        'symbol',
        [MARKET],
        [COMMODITY],
        False,
        'https://10.99.100.116:4433/private-api/',
        'wss://10.99.100.116:4433/tm',
        '',
        '',
        ('10.99.100.116', 6102)
    )
    client.token = TOKEN
    
    await client.login()
    await client.connect()
    client.ws_task = asyncio.create_task(client.ws_loop())
    await client.shakehand()
    
    ret = await client.save_by_symbol()
    data = ret[1][1]
    
    client.stop()
    await client.join()
    
    return data

# Fetch data
data = await fetch_captain_data()
print(f'Fetched {len(data)} bars')

[2025-11-13 21:52:21][INFO]Connection lost
[2025-11-13 21:52:21][INFO][None]
Fetched 0 bars


## Data Processing

In [ ]:
# Convert to DataFrame
df = pd.DataFrame(data)
df['datetime'] = pd.to_datetime(df['time_tag'], unit='ms')
df = df.sort_values('datetime').reset_index(drop=True)

# Calculate derived metrics
df['returns'] = df['nv'].pct_change()
df['cumulative_return'] = (1 + df['returns']).cumprod()

print(f'Data range: {df["datetime"].min()} to {df["datetime"].max()}')
print(f'Total bars: {len(df)}')
df.head()

## Visualization 1: P&L Curve

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

# Portfolio value
ax1.plot(df['datetime'], df['pv'], label='Portfolio Value (PV)', linewidth=2)
ax1.axhline(y=10000000, color='gray', linestyle='--', alpha=0.5, label='Initial Capital')
ax1.set_ylabel('Portfolio Value (¥)', fontsize=12)
ax1.set_title('Trinity Captain - Portfolio Value Over Time', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Net value (performance)
ax2.plot(df['datetime'], df['nv'], label='Net Value (NV)', color='green', linewidth=2)
ax2.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, label='Breakeven')
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('Net Value (Performance)', fontsize=12)
ax2.set_title('Trinity Captain - Net Value (Normalized Performance)', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
filename = f'WOS_Captain_pnl_{START_DATE}_{END_DATE}.png'
plt.savefig(filename, dpi=150)
print(f'Saved: {filename}')
plt.show()

## Visualization 2: Drawdown Analysis

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

# Drawdown over time
ax1.fill_between(df['datetime'], 0, -df['drawdown'] * 100, 
                 color='red', alpha=0.3, label='Drawdown')
ax1.set_ylabel('Drawdown (%)', fontsize=12)
ax1.set_title('Trinity Captain - Drawdown Over Time', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Peak tracking
ax2.plot(df['datetime'], df['peak_nv'], label='Peak NV', linewidth=2)
ax2.plot(df['datetime'], df['nv'], label='Current NV', alpha=0.7)
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('Net Value', fontsize=12)
ax2.set_title('Peak vs Current Net Value', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
filename = f'WOS_Captain_drawdown_{START_DATE}_{END_DATE}.png'
plt.savefig(filename, dpi=150)
print(f'Saved: {filename}')
plt.show()

## Visualization 3: Regime Distribution & Activity

In [ ]:
fig = plt.figure(figsize=(15, 10))
gs = fig.add_gridspec(2, 2)

# Regime distribution (pie chart)
ax1 = fig.add_subplot(gs[0, 0])
total_river = df['river_trades'].iloc[-1]
total_lake = df['lake_trades'].iloc[-1]
total_dead = df['dead_zone_count'].iloc[-1]
regime_data = [total_river, total_lake, total_dead]
ax1.pie(regime_data, labels=['River (Trend)', 'Lake (Range)', 'Dead Zone'], 
        autopct='%1.1f%%', startangle=90, colors=['blue', 'green', 'gray'])
ax1.set_title('Regime Distribution', fontsize=12, fontweight='bold')

# Active positions over time
ax2 = fig.add_subplot(gs[0, 1])
ax2.plot(df['datetime'], df['active_positions'], linewidth=1.5)
ax2.set_ylabel('Count', fontsize=10)
ax2.set_title('Active Positions Over Time', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Risk events
ax3 = fig.add_subplot(gs[1, :])
ax3.plot(df['datetime'], df['portfolio_dd_events'], label='Portfolio DD Events', linewidth=2)
ax3.plot(df['datetime'], df['basket_dd_events'], label='Basket DD Events', linewidth=2)
ax3.set_xlabel('Date', fontsize=10)
ax3.set_ylabel('Cumulative Events', fontsize=10)
ax3.set_title('Risk Management Events', fontsize=12, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
filename = f'WOS_Captain_regimes_{START_DATE}_{END_DATE}.png'
plt.savefig(filename, dpi=150)
print(f'Saved: {filename}')
plt.show()

## Performance Statistics

In [ ]:
# Calculate statistics
final_nv = df['nv'].iloc[-1]
total_return = (final_nv - 1) * 100
max_dd = df['drawdown'].max() * 100

# Sharpe ratio (annualized)
returns = df['returns'].dropna()
sharpe = np.sqrt(252 * 96) * returns.mean() / returns.std() if returns.std() > 0 else 0

# Win rate
wins = (returns > 0).sum()
total = len(returns)
win_rate = wins / total * 100 if total > 0 else 0

# Display
stats = f'''
=== TRINITY CAPTAIN PERFORMANCE ===

Final Net Value: {final_nv:.4f}
Total Return: {total_return:.2f}%
Max Drawdown: {max_dd:.2f}%
Sharpe Ratio: {sharpe:.2f}
Win Rate: {win_rate:.1f}%

Regime Distribution:
  River Trades: {total_river}
  Lake Trades: {total_lake}
  Dead Zone: {total_dead}

Risk Events:
  Portfolio DD Events: {df['portfolio_dd_events'].iloc[-1]}
  Basket DD Events: {df['basket_dd_events'].iloc[-1]}

Data Range: {df['datetime'].min()} to {df['datetime'].max()}
Total Bars: {len(df)}
'''

print(stats)

# Save to file
with open(f'WOS_Captain_summary_{START_DATE}_{END_DATE}.txt', 'w') as f:
    f.write(stats)
print('Saved summary to file')